### 1.1. Импортируем модули и начинаем работать с https://stihi.ru/:

In [1]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd

In [2]:
stihi_ru = "https://stihi.ru/"
response = requests.get("https://stihi.ru/")

In [3]:
html_content = response.text

### 1.2. Вытаскиваем ссылки из верхнего меню:

In [4]:
soup = BeautifulSoup(html_content, 'html.parser')  
post = soup.find('ul', {'class': 'headmenu'})

In [5]:
links = []
for link in post.find_all('a'):
    actualink = link.get('href')
    if "https" not in actualink:
        full_link = stihi_ru+actualink
        links.append(full_link)
    else:
        links.append(actualink)
        
links

['https://stihi.ru//authors/',
 'https://stihi.ru//poems/',
 'https://stihi.ru//board/',
 'https://stihi.ru//search.html',
 'https://shop.stihi.ru/',
 'https://o.stihi.ru/',
 'https://stihi.ru//login/']

### 2.1. Начинаем работу с ссылкой на рекомендованные произведения:

In [6]:
reclit_line = soup.find('a', {'class': 'topref'})
reclit_link = reclit_line.get('href')
reclit_link = stihi_ru+reclit_link
reclit_link

'https://stihi.ru//editor/'

In [7]:
response_2 = requests.get(reclit_link)

In [8]:
html_content_2 = response_2.text

In [9]:
soup_2 = BeautifulSoup(html_content_2, 'html.parser') 

### 2.2. Достаем и кладем в отдельные списки произведения, авторов и время публикаций

In [10]:
works = []
works_lines = soup_2.find_all('a', {'class':'poemlink'})
for work in works_lines:
    works.append(work.text)

In [11]:
autors = []
autors_lines = soup_2.find_all('a', {'class':'authorlink'})
for autor in autors_lines:
    autors.append(autor.text)

In [12]:
dates = []
dates_lines = soup_2.find_all('small')
for date in dates_lines:
    date_text = date.text   
    date_text = date_text.replace('-', '')
    dates.append(date_text)


In [13]:
YMDs = []
for date in dates:
    year_month_date_time = date.split()
    year_month_date = year_month_date_time[0]
    y_m_d = year_month_date.split('.')
    YMD = f"{y_m_d[2]}-{y_m_d[1]}-{y_m_d[0]}"
    YMDs.append(YMD)

In [14]:
data = {
    "Автор": autors,
    "Произведение": works,
    "Время публикации": YMDs
}

In [19]:
df = pd.DataFrame(data).sort_values(by="Время публикации")
df

,Автор,Произведение,Время публикации
10,Елена Заостровцева,Бархатный сезон,2021-09-22
23,Александр Забродин 3,Дождь небольшой,2021-10-01
21,Игумен Паисий Савосин,Свой мир,2021-10-01
20,Нина Баландина,Каждая жизнь - отражение лунных зеркал,2021-10-01
19,Хубулава Григорий Геннадьевич,Октября дрожащего костёр,2021-10-01
18,Марианна Боровкова,Была бы я женщиной,2021-10-01
17,Катя Волкова,Тускло светит неба матовый абажур,2021-10-01
16,Фокеева Елена,Начало осени,2021-10-01
22,Юрий Рымзин,Ведунья,2021-10-01
15,Вера Сергеевна Бутко,Дед,2021-10-01


#### Запишем все в файл:

In [22]:
writer = pd.ExcelWriter('stihi_data.xlsx', engine='xlsxwriter')
   
df.to_excel(writer, 'Sheet1', index=False)

writer.save()